In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn.functional as F

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import gym

In [4]:
from gym import Env
from gym.spaces import Discrete, Box
import random


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [64]:
import numpy as np
path_to_data = "/gdrive/MyDrive/MIE1666project/rl-ccmvo-main/rl-ccmvo-main/Data/synthetic1000"
date = "2016-02-07"
mu = np.loadtxt('{}/{}/mu'.format(path_to_data, date))
Q = np.loadtxt('{}/{}/Q'.format(path_to_data, date))

# Normalization of Q and mu

In [65]:
import torch
Q/=torch.linalg.norm(torch.tensor(Q))
mu/=torch.linalg.norm(torch.tensor(mu))

# Three important function for the RL env set-up, **MVO(mu,Q,gamma), generate_Q(Q,index) and derivative(mu,Q,gamma,w)**


*   MVO(mu,Q,gamma) returns the weight vector and the negative MVO (return-risk) value.
*   generate_Q(Q,index) is used in each time step for the agent to generate a **Q** convariance for allocated k assets.
*   derivative(mu,Q,gamma,w) is used to give the derivative MVO which is used to provide the reward flag for +1 if derivative >0 or else reward=-1.




In [7]:
import cvxpy as cp
def MVO(mu,Q,gamma):
    # Convert mu to NumPy array if it's not already
  if not isinstance(mu, np.ndarray):
      mu = np.array(mu)

  # Convert Q to NumPy array if it's not already
  if not isinstance(Q, np.ndarray):
      Q = np.array(Q)
  if mu.size == 0 or Q.size == 0:
    return 0,0
  else:
    n = len(mu)

    # Decision Variables
    w = cp.Variable(n)

    constraints = [
        cp.sum(w) == 1, # Sum to 1
        w>=0 # Disallow Short Sales
    ]

    # Objective Function
    risk = cp.quad_form(w, Q)
    targetRet=gamma*mu.T@w
    prob = cp.Problem(cp.Maximize(targetRet - risk), constraints=constraints)
    prob.solve()
    return w.value,targetRet.value-risk.value

In [8]:
# function to select Qs from the Q array of 1000/500 el
# def generate_Q(Q,index):
#   # Use np.meshgrid to generate all pairs of indices
#   all_pairs = np.array(np.meshgrid(index,index)).T.reshape(-1, 2)

#   # Use advanced indexing to select the elements
#   selected_elements = Q[all_pairs[:, 0], all_pairs[:, 1]]
#   size=int(np.sqrt(selected_elements.size))
#   return selected_elements.reshape(size,size)
def generate_Q(Q, index):
    # Convert index to a PyTorch tensor
    index_tensor = torch.tensor(index)

    # Use torch.meshgrid to generate all pairs of indices
    all_pairs = torch.meshgrid(index_tensor, index_tensor)
    all_pairs = torch.stack(all_pairs, dim=-1).reshape(-1, 2)

    # Use advanced indexing to select the elements
    selected_elements = Q[all_pairs[:, 0], all_pairs[:, 1]]

    # Calculate the size based on the length of the index array
    size = len(index)

    return selected_elements.reshape(size, size)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# derivative function always return the gradient of MVO wrt the last x added to the list
def derivative(mu,Q,gamma,w):
  # if mu.size <= 1 or Q.size <= 1:
  #   return -gamma*mu[0]
  # else:
  risk=0
  for i in range(len(mu)):
      risk+=Q[i][-1]*w[i]
  Ret=mu[-1]*gamma
      # Ret+=mu[-1]*gamma
  return Ret-risk


# Let's build Our Environment

In [57]:
class TrainEnv(Env):
## initialize parameter:##

    def __init__(self, Q):

        self.index = [] #index of the latest asset added to allocation
        self.cur_mu = [] #cur_mu and cur_Q at time step k
        self.cur_Q = []
        self.action_space = Discrete(500) # action_space and observation _space are defined, action space should be 1000, e.g 0 means the first asset is selected
        self.observation_space = Box(low=-np.inf, high=np.inf)
        # state is a numpy array float
        self.state =np.array([0.]) # Initialize state which should be state[0]= MVO if updated
        self.asset_length = 50 # asset length to end
        self.used_actions = set()  # Keep track of used actions in the current iteration

    def step(self, action, done):
        if done == True:
          self.used_actions=set()
          self.index = [] #index of the latest asset added to allocation
          self.cur_mu = [] #cur_mu and cur_Q at time step k
          self.cur_Q = []
        # action is the index of the asset
        action = self.sample_unique_action()
        self.used_actions.add(action)
        gamma = 1
        self.index.append(action)
        self.cur_mu.append(mu[action])
        self.cur_Q=generate_Q(Q,self.index) # Assuming generate_Q is defined
        # print("cur mu:",len(self.cur_mu),"cur_Q",self.cur_Q.size())
        w, self.state[0] = MVO(self.cur_mu, self.cur_Q, gamma)  # Assuming MVO is defined
        self.asset_length -= 1
        self.transition = derivative(self.cur_mu,self.cur_Q, gamma, w)  # Assuming derivative is defined

        # Calculate reward
        reward=self.transition.item()
        # if self.transition > 0:
        #     reward = 1
        # else:
        #     reward = -1

        done = self.asset_length <= 0

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    # Make sure each action are unique
    def sample_unique_action(self):
      # Sample a new action until it is unique in the current iteration
      new_action = self.action_space.sample()
      while new_action in self.used_actions:
          new_action = self.action_space.sample()
      return new_action
    # def render(self):
    #     # Implement viz
    #     pass

    def reset(self):
        # Reset
        self.state = np.array([0.])
        self.used_actions.clear()
        self.asset_length = 50
        return self.state


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [66]:
env=TrainEnv(Q)

In [12]:
env.reset()

array([0.])

In [13]:
action=env.action_space.sample()
action

458

# DQN model (not finished)
We will need two Neural network in DQN, one is for evaluation and one is for forward pass (Target).

In [61]:
# Hyper Parameters
BATCH_SIZE = 32
LR = 0.01                   # learning rate
EPSILON = 0.8               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 100   # target update frequency
MEMORY_CAPACITY = 2000 # Memory for experience replay
# env = gym.make('CartPole-v0')
# env = env.unwrapped
N_ACTIONS = env.action_space.n
N_STATES = env.observation_space.shape[0]
ENV_A_SHAPE = 0 if isinstance(env.action_space.sample(), int) else env.action_space.sample().shape

In [16]:
N_ACTIONS

500

In [45]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(N_STATES, 10)
        self.fc1.weight.data.normal_(0, 0.1)   # initialization
        self.out = nn.Linear(10, N_ACTIONS)
        self.out.weight.data.normal_(0, 0.1)   # initialization

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value

In [18]:
# eval_net=Net()
# x=env.reset()
# x= torch.unsqueeze(torch.FloatTensor(x), 0)
# actions_value = eval_net.forward(x)
# for i in used_action:
#     actions_value[0][i] = -10000
# action = torch.max(actions_value, 1)[1].data.numpy()
# action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index
# used_action.add(action)
# torch.max(actions_value, 1)[1].data.numpy()

In [62]:
class DQN(object):
    def __init__(self):
        self.eval_net, self.target_net = Net(), Net()
        self.learn_step_counter = 0                                     # for target updating
        self.memory_counter = 0                                         # for storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))     # initialize memory
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)
        self.loss_func = nn.MSELoss()
        self.used_actions = set()  # Keep track of used actions in the current iteration


    def choose_action(self,x,done):
        x = torch.unsqueeze(torch.FloatTensor(x), 0)
        # input only one sample
        if done == True:
          self.used_actions=set()
        if np.random.uniform() < EPSILON:   # greedy
            # print("it's greedy")
            actions_value = self.eval_net.forward(x)
            for i in self.used_actions:
                actions_value[0][i] = -1000 #assign a large negative value for selected action such that it wont select it again
            action = torch.max(actions_value, 1)[1].data.numpy()
            action = action[0] if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)  # return the argmax index

        else:   # random
            # print("it's random")
            available_actions = list(set(range(N_ACTIONS))- self.used_actions)
            action = np.random.choice(available_actions)
            action = action if ENV_A_SHAPE == 0 else action.reshape(ENV_A_SHAPE)
        self.used_actions.add(action)


        # print("used action:",self.used_actions)
        # print("Used action:",self.used_actions,"action for this iter:",action, "ENV_A_SHAPE:",ENV_A_SHAPE)
        return action

    def store_transition(self, s, a, r, s_):
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % MEMORY_CAPACITY
        self.memory[index, :] = transition
        self.memory_counter += 1
    def learn(self):
        # target parameter update
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
        self.learn_step_counter += 1

        # sample batch transitions
        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)
        b_memory = self.memory[sample_index, :]
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        # q_eval w.r.t the action in experience
        q_eval = self.eval_net(b_s).gather(1, b_a)  # shape (batch, 1)
        q_next = self.target_net(b_s_).detach()     # detach from graph, don't backpropagate
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)   # shape (batch, 1)
        loss = self.loss_func(q_eval, q_target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [22]:
dqn = DQN()

In [67]:
dqn = DQN()
done=False
print('\nCollecting experience...')
for i_episode in range(200):
    s = env.reset()
    ep_r = 0
    while True:
        a = dqn.choose_action(s,done)
        s_, r, done, info= env.step(a,done)
        # if done ==True:
        #   s=env.reset()
        # modify the reward
        # x, x_dot, theta, theta_dot = s_
        # r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        # r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        # r = r1 + r2
        dqn.store_transition(s, a, r, s_)
        ep_r += r
        # print(ep_r,done,dqn.memory_counter)
        if dqn.memory_counter > MEMORY_CAPACITY:
            # print('Learning')
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_


Ep:  40 | Ep_r:  1.1
Ep:  41 | Ep_r:  1.23
Ep:  42 | Ep_r:  1.19
Ep:  43 | Ep_r:  1.32
Ep:  44 | Ep_r:  1.33
Ep:  45 | Ep_r:  1.43
Ep:  46 | Ep_r:  1.36
Ep:  47 | Ep_r:  1.13
Ep:  48 | Ep_r:  1.14
Ep:  49 | Ep_r:  1.06
Ep:  50 | Ep_r:  1.23
Ep:  51 | Ep_r:  1.38
Ep:  52 | Ep_r:  1.23
Ep:  53 | Ep_r:  1.06
Ep:  54 | Ep_r:  1.11
Ep:  55 | Ep_r:  1.19
Ep:  56 | Ep_r:  1.18
Ep:  57 | Ep_r:  1.15
Ep:  58 | Ep_r:  1.09
Ep:  59 | Ep_r:  1.26
Ep:  60 | Ep_r:  1.14
Ep:  61 | Ep_r:  1.37
Ep:  62 | Ep_r:  1.44
Ep:  63 | Ep_r:  1.1
Ep:  64 | Ep_r:  0.93
Ep:  65 | Ep_r:  1.03
Ep:  66 | Ep_r:  1.47
Ep:  67 | Ep_r:  1.09
Ep:  68 | Ep_r:  1.31
Ep:  69 | Ep_r:  1.16
Ep:  70 | Ep_r:  1.15
Ep:  71 | Ep_r:  1.26
Ep:  72 | Ep_r:  1.29
Ep:  73 | Ep_r:  1.23
Ep:  74 | Ep_r:  1.31
Ep:  75 | Ep_r:  1.36
Ep:  76 | Ep_r:  1.1
Ep:  77 | Ep_r:  1.13
Ep:  78 | Ep_r:  1.39
Ep:  79 | Ep_r:  1.43
Ep:  80 | Ep_r:  1.17
Ep:  81 | Ep_r:  1.41
Ep:  82 | Ep_r:  1.17
Ep:  83 | Ep_r:  1.26
Ep:  84 | Ep_r:  1.3
Ep:  85 | Ep_

In [ ]:

dqn = DQN()

print('\nCollecting experience...')
for i_episode in range(400):
    s = env.reset()
    ep_r = 0
    while True:
        a = dqn.choose_action(s)
        s_, r, done, info= env.step(a)
        print(a,s_,r,done,ep_r)
        # modify the reward
        # x, x_dot, theta, theta_dot = s_
        # r1 = (env.x_threshold - abs(x)) / env.x_threshold - 0.8
        # r2 = (env.theta_threshold_radians - abs(theta)) / env.theta_threshold_radians - 0.5
        # r = r1 + r2

        dqn.store_transition(s, a, r, s_)

        ep_r += r
        if dqn.memory_counter > MEMORY_CAPACITY:
            dqn.learn()
            if done:
                print('Ep: ', i_episode,
                      '| Ep_r: ', round(ep_r, 2))

        if done:
            break
        s = s_

In [ ]:
# episodes = 50
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0

#     while not done:
#         #env.render()
#         action = env.action_space.sample()
#         n_state, reward, done, info= env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'.format(episode, score))

In [ ]:
!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# pip install "ray[rllib]" tensorflow torch

In [ ]:
# from ray.rllib.algorithms.dqn.dqn import DQNConfig

In [ ]:
# config = DQNConfig()
# print(config.replay_buffer_config)
# replay_config = config.replay_buffer_config.update(
#     {
#         "capacity": 60000,
#         "prioritized_replay_alpha": 0.5,
#         "prioritized_replay_beta": 0.5,
#         "prioritized_replay_eps": 3e-6,
#     }
# )
# config = config.training(replay_buffer_config=replay_config)
# config = config.resources(num_gpus=1)
# config = config.rollouts(num_rollout_workers=3)
# algo = config.build()
# algo.train()